In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from optimus import Optimus

In [5]:
# Create optimus
op = Optimus(master="local", app_name= "optimus")


             ____        __  _                     
            / __ \____  / /_(_)___ ___  __  _______
           / / / / __ \/ __/ / __ `__ \/ / / / ___/
          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
              /_/                                  
              
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder ( local ). If you are in a cluster initialize optimus with master='your_ip' as param
Deleting previous folder if exists...
Creating the checkpoint directory...
Optimus successfully imported. Have fun :).


In [6]:
df = op.read.csv("Meteorite_Landings.csv", header=True)

In [7]:
df.show()

+-------------------+---+--------+------------+--------+----+--------------------+----------+-----------+--------------------+
|               name| id|nametype|    recclass|mass (g)|fall|                year|    reclat|    reclong|         GeoLocation|
+-------------------+---+--------+------------+--------+----+--------------------+----------+-----------+--------------------+
|             Aachen|  1|   Valid|          L5|      21|Fell|01/01/1880 12:00:...| 50.775000|   6.083330|(50.775000, 6.083...|
|             Aarhus|  2|   Valid|          H6|     720|Fell|01/01/1951 12:00:...| 56.183330|  10.233330|(56.183330, 10.23...|
|               Abee|  6|   Valid|         EH4|  107000|Fell|01/01/1952 12:00:...| 54.216670|-113.000000|(54.216670, -113....|
|           Acapulco| 10|   Valid| Acapulcoite|    1914|Fell|01/01/1976 12:00:...| 16.883330| -99.900000|(16.883330, -99.9...|
|            Achiras|370|   Valid|          L6|     780|Fell|01/01/1902 12:00:...|-33.166670| -64.950000|(-33.1

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType
df = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True),
                ("bool", "string", True)

            ]
,
[
                ("  I like     fish  ", 1, "dog", "housé", "cat-car", "a","1", "True"),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2", "True"),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3", "False"),
                (None, 3, "eagle", "glass", "lion-pc", "c","4", "False"),
                (None, 3, "eagle", "glass", "lion-pc", "True",1, False)
            ])

In [9]:
profiler = op.profiler(df)

+-------------------+---+-------+-----+-------------+------+-----+-----+
|              words|num|animals|thing|  two strings|filter|num 2| bool|
+-------------------+---+-------+-----+-------------+------+-----+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1| True|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2| True|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|False|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|False|
|               null|  3|  eagle|glass|      lion-pc|  True|    1|false|
+-------------------+---+-------+-----+-------------+------+-----+-----+



In [108]:
profiler.columns(["filter","num"])

{'columns': {'filter': {'uniques_count': 5,
   'p_uniques': 100.0,
   'missing_count': 0,
   'p_missing': 0.0,
   'frequency': [{'value': 3, 'count': 2, 'percentage': 40.0},
    {'value': 2, 'count': 2, 'percentage': 40.0},
    {'value': 1, 'count': 1, 'percentage': 20.0}]},
  'num': {'uniques_count': 3,
   'p_uniques': 60.0,
   'missing_count': 0,
   'p_missing': 0.0,
   'frequency': [{'value': 3, 'count': 2, 'percentage': 40.0},
    {'value': 2, 'count': 2, 'percentage': 40.0},
    {'value': 1, 'count': 1, 'percentage': 20.0}]}},
 'rows_count': 5}

In [ ]:
profiler.dataset_info()

In [ ]:
profiler.count_data_types('*')

In [ ]:
profiler.columns_by_types('*')

In [ ]:
df =df.cols().cast([("reclat", "integer"),("reclong","integer")])

from pyspark.ml.feature import Bucketizer
splits = [-float("inf"), -0.5, 0.0, 0.5, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="reclat", outputCol="reclat_features")
bucketedData = bucketizer.transform(df)

print("Bucketizer output with %d buckets" % (len(bucketizer.getSplits())-1))
bucketedData.show(5)